<a href="https://colab.research.google.com/github/aekanun2020/2022-PEA-ML-sklearn-Spark-/blob/main/MinimumCode_ScikitLearn_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
! wget https://storage.googleapis.com/2022oct23/LoanStats_web.csv

--2022-11-04 00:40:55--  https://storage.googleapis.com/2022oct23/LoanStats_web.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.128, 142.251.45.16, 172.217.0.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1160243241 (1.1G) [text/csv]
Saving to: ‘LoanStats_web.csv’

LoanStats_web.csv   100%[===================>]   1.08G  63.8MB/s    in 18s     

2022-11-04 00:41:14 (60.7 MB/s) - ‘LoanStats_web.csv’ saved [1160243241/1160243241]



In [3]:
loans = pd.read_csv('LoanStats_web.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,19,55,112,123,124,125,128,129,130,133,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
## เลือกเฉพาะ rows ข้อมูลที่มี loan_staus เป็น Fully Paid และ Charged Off

loans = loans[loans['loan_status'].isin(['Fully Paid', 'Charged Off'])]

In [5]:
selectedCol_list = ['loan_amnt','int_rate','loan_status','issue_d']

In [6]:
loans = loans[selectedCol_list].sample(50000)

In [7]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 1031897 to 565640
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   loan_amnt    50000 non-null  float64
 1   int_rate     50000 non-null  object 
 2   loan_status  50000 non-null  object 
 3   issue_d      50000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.9+ MB


In [8]:
loans.shape

(50000, 4)

In [9]:
## เปลี่ยน data type ของ issue_d เป็น datetime
loans['issue_d'] = pd.to_datetime(loans['issue_d'])

In [10]:
loans['charged_off'] = (loans['loan_status'] == 'Charged Off').apply(np.uint8)
loans.drop('loan_status', axis=1, inplace=True)

In [11]:
# สร้าง training set และ testing set ซึ่งเป็นแบบ Hand-out testing set
# โดยข้อมูลที่เก่ากว่า issue_d ที่ quantile 0.9 ถูกนำมาสุ่มเลือกเพื่อเป็น training set
# ส่วนข้อมูลที่ใหม่กว่าฯ ถูกนำมาสุ่มเลือกเพื่อเป็น testing set
loans_train = loans.loc[loans['issue_d'] < loans['issue_d'].quantile(0.9)].sample(frac=0.5).dropna() 
loans_test = loans.loc[loans['issue_d'] >= loans['issue_d'].quantile(0.9)].sample(frac=0.5).dropna()

In [12]:
print(loans_train[loans_train['charged_off']==1].shape)
print(loans_train[loans_train['charged_off']==0].shape)

(4646, 4)
(17584, 4)


In [13]:
loans_train = loans_train.drop('issue_d',axis=1)
loans_test = loans_test.drop('issue_d',axis=1)

y_train = loans_train['charged_off']
y_test = loans_test['charged_off']

X_train = loans_train.drop('charged_off', axis=1)
X_test = loans_test.drop('charged_off', axis=1)

In [14]:


# รวบรวมชื่อ Column ที่เป็น String เก็บไว้ในตัวแปรชื่อ categorical
categorical = list(X_train.select_dtypes('object').columns)
print(f"Categorical columns are: {categorical}")

# รวบรวมชื่อ Column ที่เป็น numerical เก็บไว้ในตัวแปรชื่อ numerical
numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")



Categorical columns are: ['int_rate']
Numerical columns are: ['loan_amnt']


In [15]:
from sklearn.linear_model import LinearRegression, LogisticRegression

In [16]:
# กำหนด ML Algorithm
lr = LogisticRegression()

# เลือกเฉพาะ Numerical variables มาสร้างโมเดล
# train_cols = X_train[numerical].columns.tolist()

# เลือกเฉพาะ Numerical variables มาเป็น Feature
# แล้วส่ง Feature เก็บไว้ในตัวแปรชื่อ features
features = X_train[numerical]

# ส่ง Label เก็บไว้ในตัวแปรชื่อ target
target = y_train

# สร้าง ML Model
lr.fit(features, target)


LogisticRegression()

In [17]:


from sklearn.metrics import confusion_matrix

# การนำ model lr มา make prediction โดยค่าที่ predict ถูกเก็บไว้ใน y_pred
y_pred = lr.predict(X_test[numerical]).flatten()

# นำค่าที่ predict (y_pred) มาเปรียบเทียบกับค่าจริงที่อยู่ใน y_test
cm = confusion_matrix(y_test,y_pred)

# สรุปผลการเปรียบเทียบด้วย Confusion Matrix
pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)



Predicted,0,All
Actual,,
0,2308,2308
1,462,462
All,2770,2770


In [18]:
from sklearn.metrics import confusion_matrix

# นำผลการเปรียบเทียบมาคำนวณ เพื่อประเมินตามเกณฑ์
#cm = confusion_matrix(y_test, y_pred)
#print("Confusion Matrix (predict/actual):\n", pd.DataFrame(cm))
lr.score(X_test[numerical], y_test)


#เกณฑ์ recall
recall = np.diag(cm) / np.sum(cm, axis=1)
print("\nRecall:\n", pd.Series(recall))

#เกณฑ์ precision
precision = np.diag(cm) / np.sum(cm, axis=0)
print("\nPrecision:\n", pd.Series(precision))

#เกณฑ์ f1
from sklearn.metrics import f1_score
weighted_f1 = f1_score(y_test, y_pred, average="weighted")
print("\nF1 score:\n", weighted_f1)



Recall:
 0    1.0
1    0.0
dtype: float64

Precision:
 0    0.833213
1         NaN
dtype: float64

F1 score:
 0.7574066938431943


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
